In [0]:
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re
from keras import backend as K
import keras.layers as layers
from keras.models import Model, load_model
from keras.engine import Layer
import numpy as np
import json
from sklearn.model_selection import train_test_split
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())
# Initialize session
sess = tf.Session()

Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14255678887039592551
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 3543633615335135261
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 3494147271708632236
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14892338381
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5210426998820853595
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]


In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# takes equal amounts of samples from each category
# returns train test data set
def create_sample(sample_amount):
    if sample_amount > 4000:
        raise ValueError('cannot sample more then 4000 data points.')
    data = pd.read_csv('drive/My Drive/bert_rte/data/wrangled_train.csv', index_col=0)
    sampled_text = []
    sampled_id = []
    sampled_tag = []
    for tag in list(data.Tag.unique()):
        sampled = data[data['Tag'] == tag].sample(sample_amount, random_state=42)
        sampled_text_ = list(sampled['Text'])
        sampled_id_ = list(sampled['ID'])
        sampled_tag_ = list(sampled['Tag_num'])

        for item in sampled_text_:
            sampled_text.append(item)
        for item in sampled_id_:
            sampled_id.append(item)
        for item in sampled_tag_:
            sampled_tag.append(item)

    with open('sampled_data.txt', 'w') as outfile:
        for sample_index in range(len(sampled_id)):
            json.dump({sampled_id[sample_index]: [sampled_text[sample_index], sampled_tag[sample_index]]}, outfile)
            outfile.write('\n')

    with open('sampled_data.txt', 'r') as infile:
        data = [json.loads(i) for i in infile if type(list(json.loads(i).values())[0][0]) == str]
    X_train, X_test, y_train, y_test = train_test_split([list(i.values())[0][0] for i in data],
                                                        [list(i.values())[0][1] for i in data], test_size=0.20,
                                                        random_state=42)
    
    split = len(X_train)
    X = X_train + X_test
    y = y_train + y_test

    y_one_hot = []
    for i in y:
      temp = np.zeros(23)
      temp[i - 1] = 1
      y_one_hot.append(temp)

    y_one_hot = np.array(y_one_hot)
    X_train = X[:split]
    X_test = X[split:]
    y_train = y_one_hot[:split]
    y_test = y_one_hot[split:]
    return X_train, X_test, y_train, y_test

In [0]:
X_train, X_test, y_train, y_test = create_sample(2000)

In [0]:
class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable=True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/2', trainable=self.trainable,
                               name="{}_module".format(self.name))

        self.trainable_weights += K.tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result

    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

In [0]:

# Function to build model
def build_model(): 
  input_text = layers.Input(shape=(1,), dtype="string")
  embedding = ElmoEmbeddingLayer()(input_text)
  dense = layers.Dense(256, activation='relu')(embedding)
  pred = layers.Dense(23, activation='softmax')(dense)

  model = Model(inputs=[input_text], outputs=pred)

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.summary()
  
  return model

In [0]:
# Create datasets (Only take up to 150 words for memory)
train_text = [' '.join(t.split()[0:150]) for t in X_train]
train_text = np.array(train_text, dtype=object)[:, np.newaxis]

test_text = [' '.join(t.split()[0:150]) for t in X_test]
test_text = np.array(test_text, dtype=object)[:, np.newaxis]


train_labels = np.array(y_train)
test_labels = np.array(y_test)

In [0]:
# Build and fit
model = build_model()
model.fit(train_text, 
          y_train,
          validation_data=(test_text, y_test),
          epochs=4,
          batch_size=32)

W0705 03:55:57.189936 139922244118400 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0705 03:55:57.191655 139922244118400 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0705 03:55:58.235071 139922244118400 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0705 03:55:58.261150 139922244118400 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0705 03:55:58.283183 139922244118400 deprecation_wrappe

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
elmo_embedding_layer_1 (Elmo (None, 1024)              4         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_2 (Dense)              (None, 23)                5911      
Total params: 268,315
Trainable params: 268,315
Non-trainable params: 0
_________________________________________________________________


W0705 03:55:58.522406 139922244118400 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 36780 samples, validate on 9196 samples
Epoch 1/4
36780/36780 [==============================] - 874s 24ms/step - loss: 1.1149 - acc: 0.6664 - val_loss: 0.9341 - val_acc: 0.7113
Epoch 2/4
36780/36780 [==============================] - 878s 24ms/step - loss: 0.8831 - acc: 0.7222 - val_loss: 0.8671 - val_acc: 0.7258
Epoch 3/4
36780/36780 [==============================] - 877s 24ms/step - loss: 0.8161 - acc: 0.7386 - val_loss: 0.8328 - val_acc: 0.7350
Epoch 4/4
36780/36780 [==============================] - 879s 24ms/step - loss: 0.7775 - acc: 0.7480 - val_loss: 0.8337 - val_acc: 0.7306


In [0]:
predicted = model.predict(test_text)

In [0]:
np.save('elmo_predictions', predicted)